### Introduction to Big-M reformulation - Disjunctive programming
Suppose that you have 3 machines to manufacture shoes, and the cost of each machine is:
$$𝐶_1 = 2𝑛_1$$
$$𝐶_2 = 6𝑛_2$$
$$𝐶_3 = 7𝑛_3$$
where $𝐶_𝑖$ is cost for production of machine $i$, $𝑛_𝑖$ is the number of shoes manufactured in machine $i$.
Each machine has a limit of production of 1.000 shoes. For a total production of 2.100 shoes, how many shoes should each machine made in order to minimize the total cost?

**Machine 2 has a start cost of 1000.**

$$\begin{align*} \min \quad & C_1 + C_2 + C_3 \\
\mathrm{s.t.} \quad & 𝐶_1 = 2𝑛_1\\
& 𝐶_2 = 6𝑛_2 +1000y\\
& n_2 \le M*y\\
& 𝐶_3 = 7𝑛_3 \\
& n_1 + n_2 + n_3 = 2100\\
& 0 \leq n_1, n_2, n_3 \leq 1000
&& n_i \in \mathrm{integers}\\
& y \in \mathrm{binary}
\end{align*} $$

In [7]:
import pyomo.environ as pe
import pyomo.opt as po

In [8]:
m = pe.ConcreteModel()
M = 1e4

m.S = pe.RangeSet(1,3)
m.n = pe.Var(m.S, domain=pe.NonNegativeIntegers, bounds=(0,1000))
m.C = pe.Var(m.S, domain=pe.Reals)
m.y = pe.Var(domain=pe.Binary)

m.cons1 = pe.Constraint(expr=(pe.summation(m.n) == 2100)) # pe.summation sums all variables from all sets
m.cons2 = pe.Constraint(expr=(2*m.n[1] == m.C[1]))
m.cons3 = pe.Constraint(expr=((6*m.n[2] + 1000)*m.y == m.C[2]))
m.cons4 = pe.Constraint(expr=(m.n[2] <= M*m.y))
m.cons5 = pe.Constraint(expr=(7*m.n[3] == m.C[3]))

m.obj = pe.Objective(expr=pe.summation(m.C),sense=pe.minimize)

In [9]:
solver = po.SolverFactory('couenne') 
results = solver.solve(m)

print('n1', pe.value(m.n[1]))
print('n2', pe.value(m.n[2]))
print('n3', pe.value(m.n[3]))
print('Total', pe.value(pe.summation(m.n)))

print('\ny', pe.value(m.y))

print('\nC1', pe.value(m.C[1]))
print('C2', pe.value(m.C[2]))
print('C3', pe.value(m.C[3]))
print('CTotal', pe.value(pe.summation(m.C)))

n1 1000.0
n2 1000.0
n3 100.00000000000014
Total 2100.0

y 1.0

C1 2000.0
C2 7000.0
C3 700.000000000001
CTotal 9700.000000000002


----

Suppose that you have 3 machines to manufacture shoes, and the cost of each machine is:
$$𝐶_1 = 2𝑛_1$$
$$𝐶_2 = 6𝑛_2$$
$$𝐶_3 = 7𝑛_3$$
where $𝐶_𝑖$ is cost for production of machine $i$, $𝑛_𝑖$ is the number of shoes manufactured in machine $i$.
Each machine has a limit of production of 1.000 shoes. For a total production of 2.100 shoes, how many shoes should each machine made in order to minimize the total cost?

**Machine 2 can only be ON if Machine 1 is ON**

$$\begin{align*} \min \quad & C_1 + C_2 + C_3 \\
\mathrm{s.t.} \quad & 𝐶_1 = 2𝑛_1\\
& n_1 \le M*y_1\\
& 𝐶_2 = 6𝑛_2\\
& n_2 \le M*y_1*y_2\\
& 𝐶_3 = 7𝑛_3 \\
& n_1 + n_2 + n_3 = 2100\\
& 0 \leq n_1, n_2, n_3 \leq 1000
&& n_i \in \mathrm{integers}\\
& y_i \in \mathrm{binary}
\end{align*} $$

In [10]:
m = pe.ConcreteModel()
M = 1e7

m.S = pe.RangeSet(1,3)
m.n = pe.Var(m.S, domain=pe.NonNegativeIntegers, bounds=(0,1000))
m.C = pe.Var(m.S, domain=pe.Reals)
m.y1 = pe.Var(domain=pe.Binary)
m.y2 = pe.Var(domain=pe.Binary)

m.cons1 = pe.Constraint(expr=(pe.summation(m.n) == 2100))
m.cons2 = pe.Constraint(expr=(2*m.n[1] == m.C[1]))
m.conss2 = pe.Constraint(expr=(m.n[1] <= M*m.y1))
m.cons3 = pe.Constraint(expr=(6*m.n[2] == m.C[2]))
m.cons4 = pe.Constraint(expr=(m.n[2] <= M*m.y1*m.y2))
m.cons5 = pe.Constraint(expr=(7*m.n[3] == m.C[3]))

m.obj = pe.Objective(expr=pe.summation(m.C),sense=pe.minimize)


solver = po.SolverFactory('couenne') 
results = solver.solve(m)

print('n1', pe.value(m.n[1]))
print('n2', pe.value(m.n[2]))
print('n3', pe.value(m.n[3]))
print('Total', pe.value(pe.summation(m.n)))

print('\ny1', pe.value(m.y1))
print('y2', pe.value(m.y2))

print('\nC1', pe.value(m.C[1]))
print('C2', pe.value(m.C[2]))
print('C3', pe.value(m.C[3]))
print('CTotal', pe.value(pe.summation(m.C)))

n1 1000.0
n2 999.9999909999849
n3 100.0000090000151
Total 2100.0

y1 1.0
y2 1.0

C1 2000.0
C2 5999.99994599991
C3 700.0000630001057
CTotal 8700.000009000016


In this case the problem can be transformed to a linear problem, adding a new variable $z$

$$\begin{align*} \min \quad & C_1 + C_2 + C_3 \\
\mathrm{s.t.} \quad & 𝐶_1 = 2𝑛_1\\
& n_1 \le M*y_1\\
& 𝐶_2 = 6𝑛_2\\
& n_2 \le M*z\\
& z \le y_1\\
& z \le y_2\\
& z \ge y_1 + y_2 -1 \\
& 𝐶_3 = 7𝑛_3 \\
& n_1 + n_2 + n_3 = 2100\\
& 0 \leq n_1, n_2, n_3 \leq 1000
&& n_i \in \mathrm{integers}\\
& y_i, z \in \mathrm{binary}
\end{align*} $$

In [11]:
m = pe.ConcreteModel()
M = 1e7

m.S = pe.RangeSet(1,3)
m.n = pe.Var(m.S, domain=pe.NonNegativeIntegers, bounds=(0,1000))
m.C = pe.Var(m.S, domain=pe.Reals)
m.y1 = pe.Var(domain=pe.Binary)
m.y2 = pe.Var(domain=pe.Binary)
m.z = pe.Var(domain=pe.Binary)

m.cons1 = pe.Constraint(expr=(pe.summation(m.n) == 2100))
m.cons2 = pe.Constraint(expr=(2*m.n[1] == m.C[1]))
m.conss2 = pe.Constraint(expr=(m.n[1] <= M*m.y1))
m.conss2z = pe.Constraint(expr=(m.z <= m.y1))
m.cons3 = pe.Constraint(expr=(6*m.n[2] == m.C[2]))
m.cons4 = pe.Constraint(expr=(m.n[2] <= M*m.z))
m.conss4 = pe.Constraint(expr=(m.z <= m.y2))
m.conss4z = pe.Constraint(expr=(m.z >= m.y1 + m.y2 - 1))
m.cons5 = pe.Constraint(expr=(7*m.n[3] == m.C[3]))
m.consos1 = pe.Constraint() 

m.obj = pe.Objective(expr=pe.summation(m.C),sense=pe.minimize)


solver = po.SolverFactory('glpk') 
results = solver.solve(m)

print('n1', pe.value(m.n[1]))
print('n2', pe.value(m.n[2]))
print('n3', pe.value(m.n[3]))
print('Total', pe.value(pe.summation(m.n)))

print('\ny1', pe.value(m.y1))
print('y2', pe.value(m.y2))
print('z', pe.value(m.z))

print('\nC1', pe.value(m.C[1]))
print('C2', pe.value(m.C[2]))
print('C3', pe.value(m.C[3]))
print('CTotal', pe.value(pe.summation(m.C)))

n1 1000.0
n2 1000.0
n3 100.0
Total 2100.0

y1 1.0
y2 1.0
z 1.0

C1 2000.0
C2 6000.0
C3 700.0
CTotal 8700.0
